<a href="https://colab.research.google.com/github/simonbustamante/mit-sistemas-de-recomendaciones/blob/master/Caso_de_estudio_4_1_Sistema_de_recomendaci%C3%B3n_Avanzado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caso de Estudio 4.1 - Sistema de recomendación de películas

#### Nota: Si en algún momento cierra este notebook tendrá que volver a ejecutar todas las celdas de nuevo al abrirlo.

#### Nota: Puede que obtenga diferentes resultados numéricos al ejecutar el notebook en diferentes ocasiones. Esto es normal, simplemente entregue los resultados obtenidos.

## PYTHON AVANZADO

Como esta es la versión avanzada, no incluimos parte del código en ella. Si se atasca en alguna sección del caso de estudio no dude en consultar la [versión principiante](https://drive.google.com/file/d/1rLjIyCxUSL_E7rLBFfa3NYk7sbc9tpoZ/view?usp=sharing) del caso para obtener ayuda.

## Información de contacto

In [ ]:
# TU NOMBRE = ...
# TU CORREO ELECTRÓNICO DE MITX PRO = ...

## Configuración

Ejecute (Run) estas celdas para instalar los paquetes necesarios para completar el caso de estudio. Esto podría llevar unos minutos así que sea paciente.

In [ ]:
!pip install --upgrade pip
!pip install surprise==0.1
print('Librerías instaladas con éxito!')

Si no ha obtenido ningún texto en rojo indicando error entonces la instalación ha terminado con éxito. El texto en amarillo son avisos, no errores.

<h1>Atención:</h1>

Ahora ha de reiniciar el entorno de ejecución. Para ello vaya a:

> Entorno de ejecución > _Reiniciar entorno de ejecución_

en la parte superior de su pantalla. Esto asegurará que sus cambio se han realizado con éxito.

## Importar

Importe las librerías necesarias para el desarrollo del caso.

In [ ]:
import pandas as pd
import matplotlib
from surprise import Dataset, SVD, NormalPredictor, BaselineOnly, KNNBasic, NMF
from surprise.model_selection import cross_validate, KFold
%matplotlib inline
print('Librerías importadas con éxito!')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Datos

Use la función [`Dataset.load_builtin`](http://surprise.readthedocs.io/en/stable/dataset.html#surprise.dataset.Dataset.load_builtin) para cargar la base de datos `ml-100k` de *MovieLens*.

In [ ]:
# Inserte aquí su código para cargar los datos...

También queremos tener cierta idea de la apariencia de los datos. Para ello cree un histograma de todas las valoraciones que hay en la base de datos.

In [ ]:
# Escribe aquí tu código para crear un histograma de las puntuaciones...

<h1>PREGUNTA 1: ANÁLISIS DE DATOS</h1>

**Describa la base de datos. ¿Cuántas valoraciones (ratings) hay en la base de datos? Como describiría la distribución de las valoraciones? ¿Hay algo más en lo que deberíamos fijarnos?**

Asegúrese de que el histograma se puede visualizar en el *notebook* incluso una vez se haya convertido a formato `.pdf`.

*Escribe aquí tu respuesta...*

## Modelo 1: aleatorio

In [ ]:
# Create un modelo utilizando la clase NormalPredictor().

In [ ]:
# Entrena el modelo con los datos utilizando validación cruzada con k=5 iteraciones, midiendo el RMSE
# Observa la función cross_validate que importamos al principio
# http://surprise.readthedocs.io/en/stable/model_selection.html#surprise.model_selection.validation.cross_validate

## Modelo 2: filtrado colaborativo basado en usuarios

In [ ]:
# Crea un modelo utilizando la clase KNNBasic()
# Consulta el parámetro sim_options para determinar la similitud usuario/item calculada por el modelo
# http://surprise.readthedocs.io/en/stable/prediction_algorithms.html#similarity-measures-configuration

In [ ]:
# Entrena el modelo utilizando el mismo esquema de validación cruzada
# utilizado anteriormente

## Modelo 3: filtrado colaborativo basado en ítems

In [ ]:
# Crea un modelo utilizando la clase KNNBasic()
# Asegúrate de cambiar el parámetro sim_options del código anterior

In [ ]:
# Entrena el modelo utilizando el mismo esquema de validación cruzada
# utilizado anteriormente

<h1>PREGUNTA 2: MODELOS DE FILTRADO COLABORATIVO</h1>

**Compare los resultados de los modelos de filtrado colaborativo basados en usuarios e ítems. ¿Qué diferencias encuentra entre ambos? ¿Qué diferencias encuentra entre dichos modelos y el modelo aleatorio? ¿Puede explicar qué podría haber motivado dichas diferencias en los resultados?**

*Escribe aquí tu respuesta...*

## Modelo 4: factorización de matriz

In [ ]:
# Crea un modelo utilizando la clase SVD()

In [ ]:
# Entrena el modelo utilizando el mismo esquema de validación cruzada utilizado anteriormente

<h1>PREGUNTA 3: MODELO DE FACTORIZACIÓN DE MATRIZ</h1>

**El modelo de factorización de matriz es diferente de los modelos de filtrado colaborativo. Describa brevemente en qué consisten dichas diferencias. También, compare de nuevo el RECM respecto al resto de modelos. ¿Mejora? ¿Puede ofrecer alguna explicación de por qué mejora/empeora?**

*Escribe aquí tu respuesta...*

## Precisión and exhaustividad @`k` (*precision and recall @k*)

Queremos calcular la precisión y la exhaustividad para 2 valores de `k`: 5 y 10. Hemos incluído unas cuantas líneas de código que le ayudarán a conseguirlo.

Primero, definimos una función que toma algunas predicciones, un valor de `k` y un parámetro de umbral. Este código ha sido adaptado de la siguiente [fuente](http://surprise.readthedocs.io/en/stable/FAQ.html?highlight=precision#how-to-compute-precision-k-and-recall-k).

**Asegúrese de que ejecuta esta celda**

In [ ]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Devuelve la precision y la exhaustividad @k para cada usuario'''

    # Primero asocie las predicciones a cada usuario
    user_est_true = dict()
    for uid, _, true_r, est, _ in predictions:
        current = user_est_true.get(uid, list())
        current.append((est, true_r))
        user_est_true[uid] = current

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Ordene las valoraciones de los usuarios por su valor estimado
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Número de ítems relevantes
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Número de ítems recomendados en el top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Número de ítems relevantes y recomendados en el top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@k: proporción de ítems recomendados que son relevantes
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Exhaustividad@K: proporción de ítems relevantes que se recomiendan
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

print('\n\nFunción creada con éxito!')

A continuación, calculamos la precisión y exhaustividad @`k` = 5 y 10. Usamos validación cruzada con 5 iteraciones de nuevo para promediar los resultados a lo largo de toda la base de datos.

Sea paciente porque esto podría llevar cierto tiempo en ejecutarse.

In [ ]:
# Utiliza la función precision_recall_at_k() que acaba de ser definida
# para calcular los 16 valores numéricos.

# Consulta la función test() para obtener las predicciones que se utilizan como entrada
# para la función precision_recall_at_k():
# http://surprise.readthedocs.io/en/stable/algobase.html#surprise.prediction_algorithms.algo_base.AlgoBase.test

<h1>PREGUNTA 4: PRECISIÓN/EXHAUSTIVIDAD</h1>

**Calcule la precisión y la exhaustividad, para cada uno de los 4 modelos, con `k` = 5 y 10. Es decir, 2 x 2 x 4 = 16 valores numéricos. ¿Nota algo diferente en estos valores? ¿Algo diferente de los valores de RECM calculados anteriormente?**

Esta pregunta requiere que escriba cierto código:

*Escribe aquí tu respuesta...*

##  Top-`n` Predicciones

Finalmente, queremos ver cómo son las recomendaciones y las estimaciones de valoraciones de los usuarios.

De nuevo, definimos una función auxiliar.

In [ ]:
def get_top_n(predictions, n=10):
    '''Devuelve las top-N recomendaciones para cada usuario de un conjunto de predicciones.

    Argumentos:
        predictions(lista de objetos de predicción): lista de la predicciones,
            tal y como se obtienen del método "test" de un algoritmo
        n(int): número de recomendaciones a mostrar para cada usuario.
            Por defecto es 10.

    Salidas:
    Un diccionario donde las keys son las IDs de los usuarios y los valores son
    una lista de tuples:
        [(item id, estimación de la valoración), ...] de tamaño n.
    '''

    # Primero asocie las predicciones a cada usuario.
    top_n = dict()
    for uid, iid, true_r, est, _ in predictions:
        current = top_n.get(uid, [])
        current.append((iid, est))
        top_n[uid] = current

    # A continuación ordene las predicciones para cada usuario y obtenga las
    # n predicciones más elevadas
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

print('Función creada con éxito!')

Por último, ejecutamos esta función en cada uno de los modelos, primero entrenando en **la totalidad** de los datos disponibles, y después prediciendo los datos que faltan. Usamos `n` = 5, pero puede elegir cualquier valor razonable de n.

Esto podría llevar cierto tiempo de computación, así que sea paciente.

Pista: Use [`Dataset.build_full_trainset`](http://surprise.readthedocs.io/en/stable/dataset.html#surprise.dataset.DatasetAutoFolds.build_full_trainset) para obtener el conjunto de entrenamiento de la base de datos. Después ejecute [`Trainset.build_anti_testset`](http://surprise.readthedocs.io/en/stable/trainset.html#surprise.Trainset.build_anti_testset) para obtener el conjunto de prueba. Finalmente, use `fit` en el conjunto de entrenamiento, `test` en el conjunto de prueba, y pase el resultado a la función `get_top_n()`.

In [ ]:
# Utiliza la función get_top_n() y las pistas para obtener las recomendaciones top-n
# para un usuario específico, utilizando un valor razonable para n.

<h1>PREGUNTA 5: TOP-N PREDICCIONES</h1>

**¿Tienen sentido las top-n predicciones que ha obtenido? ¿Cuál es el valor de las valoraciones (1-5) de estas predicciones? ¿Cómo podría usar estas predicciones en la vida real si estuviera intentando construir un sistema de recomendación genérico para una compañía?**

Esta pregunta requiere que escriba cierto código:

*Escribe aquí tu respuesta...*

¡Buen trabajo! Asegúrese de que comprueba la sección **Entrega** del manual de instrucciones para terminar y entregar este caso correctamente.